# Manage TSQL databases
- Create
- Backup
- Restore
- Drop

## Check Session Informations

In [44]:
PRINT N'Server Name: ' + @@servername;
PRINT N'Instance Name: ' +@@servicename;
PRINT N'Database Name: ' + DB_NAME();
PRINT N'Host Name: ' + HOST_NAME();

## Databases Name

In [47]:
SELECT * INTO [dbo].[#temp_database_name]
FROM
(
(
	SELECT N'AdventureWorks'
	UNION ALL SELECT N'AdventureWorksDW'
	UNION ALL SELECT N'time_database'

) L0 ([name])
;

In [48]:
SELECT * FROM [dbo].[#temp_database_name];

## Add server-path mapping if needed

In [49]:
SELECT * INTO [dbo].[#temp_server_path_mapping]
FROM
(
	--LOCAL
	SELECT N'LOCAL', N'D:\database_backup\local_'

	--GERSQL
	SELECT N'GERSQL', N'G:\backup\DEV_'

	-- ENVS
	UNION ALL SELECT N'DEVSQL', N'D:\db\DEV_'
	UNION ALL SELECT N'TSTSQL', N'\\TSTAPP\db\TEST_'
	UNION ALL SELECT N'PRDSQL', N'\\PRDAPP\db\PROD_'

) L0 ([server_name], [path])
;

In [64]:
SELECT * FROM [dbo].[#temp_server_path_mapping];

## Create databases

In [53]:
USE [master];
GO

DECLARE @database_name_prefix NVARCHAR(50)
    , @database_name NVARCHAR(50)
    , @sql AS NVARCHAR(MAX)
;

SET @database_name_prefix = N'';

/* main loop on the database */

DECLARE database_cursor CURSOR 
FOR 
	SELECT @database_name_prefix + [name]
	FROM [#temp_database_name]
	;
OPEN database_cursor;
FETCH NEXT FROM database_cursor INTO @database_name;

WHILE @@FETCH_STATUS = 0   
BEGIN   
	--PRINT @database_name;
	SET @sql = N'
IF DB_ID(''' + @database_name + N''') IS NULL
BEGIN
	CREATE DATABASE ' + QUOTENAME(@database_name) + N';
	ALTER DATABASE ' + QUOTENAME(@database_name) + N' SET RECOVERY SIMPLE;
END
;
';
	--PRINT @sql;
	EXECUTE sp_executesql @sql;

	FETCH NEXT FROM database_cursor INTO @database_name;
END   
;

CLOSE database_cursor;
DEALLOCATE database_cursor;

## Backup databases

In [54]:
USE [master];
GO

DECLARE @server_name NVARCHAR(50)
	, @database_name NVARCHAR(50)
	, @path NVARCHAR(256) /* path for backup files */
	, @file_name NVARCHAR(256)
	, @file_timestamp NVARCHAR(20) /* used for file name */
	, @sql NVARCHAR(MAX)
;

--SELECT @serverName = @@SERVERNAME -- it gives a different name from the one I am expexting on AWS enviroments

SELECT @server_name = CAST(SERVERPROPERTY(N'MachineName') AS NVARCHAR(50));

--PRINT @server_name

/* select the right database backup directory using the server name */

SELECT @path = [path] 
FROM [dbo].[#temp_server_path_mapping] 
WHERE [server_name] = @server_name
;

--PRINT @path;

/* specify filename format */

SELECT @file_timestamp = CAST(CONVERT(date, GETUTCDATE()) AS NVARCHAR) + N'_' + REPLACE(REPLACE(CAST(CONVERT(time, GETUTCDATE()) AS NVARCHAR), N'.', N''), N':', N'-');

/* main loop on the databases */

DECLARE database_cursor CURSOR 
FOR 
	SELECT [name]
	FROM [dbo].[sysdatabases]
	WHERE [name] IN (SELECT [name] FROM [#temp_database_name])
	;
OPEN database_cursor;
FETCH NEXT FROM database_cursor INTO @database_name;

WHILE @@FETCH_STATUS = 0   
BEGIN   
   	SET @file_name = @path + @database_name + N'_' + @file_timestamp + N'.bak';

 	PRINT N'SHRINK FILE ' + @database_name;
   	SET @sql = N'
USE [' + @database_name + '];
DECLARE @temp_sql  NVARCHAR(MAX) = '''';
SELECT @temp_sql += ''DBCC SHRINKFILE (N'''''' + f.[name] + '''''' , 0, TRUNCATEONLY); ''
FROM [sys].[sysfiles] f
;
EXEC(@temp_sql);
';
	EXEC (@sql);
   
   	PRINT N'BACKUP DATABASE ' + @database_name + N' TO DISK = ' + @file_name;
   	BACKUP DATABASE @database_name TO DISK = @file_name;

   	FETCH NEXT FROM database_cursor INTO @database_name;
END   
;

CLOSE database_cursor;
DEALLOCATE database_cursor;

## Restore databases

In [62]:
USE [master];
GO

DECLARE @server_name NVARCHAR(50)
    , @database_name_prefix NVARCHAR(50)
    , @database_name NVARCHAR(50)
    , @path NVARCHAR(256) /* path for backup files */
    , @sql NVARCHAR(MAX)
    , @file_timestamp NVARCHAR(128) = N'2020-02-03_11-00-278' /* Set the timestamp you want to use as source for the restore */
;

--SELECT @serverName = @@SERVERNAME -- it gives a different name from the one I am expexting on AWS enviroments

SELECT @server_name = CAST(SERVERPROPERTY(N'MachineName') AS NVARCHAR(50));

--PRINT @server_name

/* select the right database backup directory using the server name */

SELECT @path = [path] 
FROM [dbo].[#temp_server_path_mapping] 
WHERE [server_name] = @server_name
;

SET @database_name_prefix = N'';

/* main loop on the database */

DECLARE database_cursor CURSOR 
FOR 
	SELECT @database_name_prefix + [name]
	FROM [#temp_database_name]
	;
OPEN database_cursor;
FETCH NEXT FROM database_cursor INTO @database_name;

WHILE @@FETCH_STATUS = 0   
BEGIN   
	--PRINT @database_name;
	SET @sql = N'
IF DB_ID(''' + @database_name + N''') IS NOT NULL
BEGIN
	RESTORE DATABASE ' + QUOTENAME(@database_name) + N'
    FROM DISK = ''' + @path + @database_name + N'_' + @file_timestamp + N'.bak''
    ;
END
;
';
	--PRINT @sql;
	EXECUTE sp_executesql @sql;

	FETCH NEXT FROM database_cursor INTO @database_name;
END   
;

CLOSE database_cursor;
DEALLOCATE database_cursor;

## Drop databases

In [52]:
USE [master];
GO

DECLARE @database_name_prefix NVARCHAR(50)
    , @database_name NVARCHAR(50)
    , @sql AS NVARCHAR(MAX)
;

SET @database_name_prefix = N'';

/* main loop on the database */

DECLARE database_cursor CURSOR 
FOR 
	SELECT @database_name_prefix + [name]
	FROM [#temp_database_name]
	;
OPEN database_cursor;
FETCH NEXT FROM database_cursor INTO @database_name;

WHILE @@FETCH_STATUS = 0   
BEGIN   
	--PRINT @database_name;
	SET @sql = N'
IF DB_ID(''' + @database_name + N''') IS NOT NULL
BEGIN
	DROP DATABASE ' + QUOTENAME(@database_name) + N';
END
;
';
	--PRINT @sql;
	EXECUTE sp_executesql @sql;

	FETCH NEXT FROM database_cursor INTO @database_name;
END   
;

CLOSE database_cursor;
DEALLOCATE database_cursor;

## Drop temp table

In [63]:
DROP TABLE [dbo].[#temp_database_name];
DROP TABLE [dbo].[#temp_server_path_mapping];

## Convert in script

To convert in script the code in this notebook run the below commands in the terminal. Remember also to comment out the sections not needed (the script contains all the code cells together):

In [4]:
jupyter nbconvert manage_databases.ipynb
mv manage_databases.txt manage_databases.sql